In [256]:
import os
import pandas as pd

In [257]:
data = pd.read_excel('/content/drive/MyDrive/reconstruction_data2.xlsx')
data.head()

,id,음식,연락처,성별,연령,구,연도,월,일,날씨,...,반반피자,불고기피자,양념치킨,오리지널피자,짜장면,짬뽕,콤비네이션피자,후라이드치킨,메뉴2,만족도
0,1,중식,105160345,남,50대,연제구,2022,1,11,맑음,...,0,0,0,0,0,1,0,0,짬뽕,0
1,2,중식,104901583,여,60대 이상,연제구,2022,1,13,맑음,...,0,0,0,0,0,1,0,0,짬뽕,1
2,3,중식,108304745,여,60대 이상,연제구,2022,1,13,맑음,...,0,0,0,0,0,1,0,0,짬뽕,4
3,4,중식,107651082,여,40대,동래구,2022,1,20,맑음,...,0,0,0,0,1,0,0,0,짜장면,1
4,5,중식,107984063,남,30대,동래구,2022,1,21,맑음,...,0,1,0,0,0,0,0,0,불고기피자,2


In [258]:
raw_data=data[['음식2','성별','연령','월','날씨','최저기온','최고기온','메뉴2']]
raw_data

,음식2,성별,연령,월,날씨,최저기온,최고기온,메뉴2
0,중식,남,50대,1,맑음,-3.3,6.7,짬뽕
1,중식,여,60대 이상,1,맑음,-3.8,2.8,짬뽕
2,중식,여,60대 이상,1,맑음,-3.8,2.8,짬뽕
3,중식,여,40대,1,맑음,-3.3,8.1,짜장면
4,피자,남,30대,1,맑음,-1.0,9.8,불고기피자
...,...,...,...,...,...,...,...,...
11650,치킨,남,30대,8,맑음,27.3,33.4,반반치킨
11651,피자,여,40대,8,맑음,28.1,34.7,불고기피자
11652,치킨,남,30대,8,맑음,23.5,31.0,반반치킨
11653,치킨,여,20대,8,맑음,24.6,32.1,양념치킨


In [259]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [260]:
raw_data_copy = raw_data.copy()
raw_data_copy.loc[:,'profile'] = raw_data[['음식2','성별', '연령', '월', '날씨', '최저기온', '최고기온']].astype(str).agg(' '.join, axis=1)
raw_data_copy

,음식2,성별,연령,월,날씨,최저기온,최고기온,메뉴2,profile
0,중식,남,50대,1,맑음,-3.3,6.7,짬뽕,중식 남 50대 1 맑음 -3.3 6.7
1,중식,여,60대 이상,1,맑음,-3.8,2.8,짬뽕,중식 여 60대 이상 1 맑음 -3.8 2.8
2,중식,여,60대 이상,1,맑음,-3.8,2.8,짬뽕,중식 여 60대 이상 1 맑음 -3.8 2.8
3,중식,여,40대,1,맑음,-3.3,8.1,짜장면,중식 여 40대 1 맑음 -3.3 8.1
4,피자,남,30대,1,맑음,-1.0,9.8,불고기피자,피자 남 30대 1 맑음 -1.0 9.8
...,...,...,...,...,...,...,...,...,...
11650,치킨,남,30대,8,맑음,27.3,33.4,반반치킨,치킨 남 30대 8 맑음 27.3 33.4
11651,피자,여,40대,8,맑음,28.1,34.7,불고기피자,피자 여 40대 8 맑음 28.1 34.7
11652,치킨,남,30대,8,맑음,23.5,31.0,반반치킨,치킨 남 30대 8 맑음 23.5 31.0
11653,치킨,여,20대,8,맑음,24.6,32.1,양념치킨,치킨 여 20대 8 맑음 24.6 32.1


In [269]:
# TF-IDF 변환
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(raw_data_copy['profile'])

In [270]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [271]:
def recommend_menu(user_index, num_recommendations=2):
    # 주어진 사용자와 다른 사용자들 간의 유사도 점수
    sim_scores = list(enumerate(cosine_sim[user_index]))

    # 유사도에 따라 사용자들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 추천 메뉴 리스트 초기화
    recommended_menus = []

    # 추천 메뉴 선택
    for i in sim_scores:
        if len(recommended_menus) < num_recommendations:
            menu = raw_data_copy['메뉴2'].iloc[i[0]]
            if menu not in recommended_menus:  # 중복 방지
                recommended_menus.append(menu)
        else:
            break

    return recommended_menus

In [273]:
import random

# 데이터셋에서 무작위로 10개의 사용자 인덱스 선택
random_user_indices = random.sample(range(len(raw_data_copy)), 10)

# 결과를 저장할 빈 리스트
results = []

# 각 사용자에 대한 추천 메뉴 수행 및 해당 정보 추출
for user_index in random_user_indices:
    # 사용자 정보 추출
    user_info = raw_data_copy.loc[user_index, ['연령', '월', '날씨', '최저기온', '최고기온']]

    # 실제 메뉴와 추천 메뉴
    actual_menu = raw_data_copy['메뉴2'][user_index]
    recommended_menus = recommend_menu(user_index)

    # 결과 데이터 프레임에 추가
    results.append({
        '사용자 인덱스': user_index,
        '연령': user_info['연령'],
        '월': user_info['월'],
        '날씨': user_info['날씨'],
        '최저기온': user_info['최저기온'],
        '최고기온': user_info['최고기온'],
        '실제 정답': actual_menu,
        '추천메뉴 1': recommended_menus[0] if len(recommended_menus) > 0 else 'N/A',
        '추천메뉴 2': recommended_menus[1] if len(recommended_menus) > 1 else 'N/A',
        #'추천메뉴 3': recommended_menus[2] if len(recommended_menus) > 2 else 'N/A'
    })

# 결과를 데이터 프레임으로 변환
result_df = pd.DataFrame(results)

# 결과 출력
result_df

,사용자 인덱스,연령,월,날씨,최저기온,최고기온,실제 정답,추천메뉴 1,추천메뉴 2
0,3776,20대,12,맑음,-2.0,7.0,간짜장,짜장면,간짜장
1,8525,20대,9,비,19.0,28.8,양념치킨,양념치킨,후라이드치킨
2,8394,40대,6,흐림,22.5,25.7,후라이드치킨,후라이드치킨,반반치킨
3,6060,50대,10,맑음,13.5,22.8,짬뽕,짜장면,짬뽕
4,10858,50대,8,맑음,25.1,32.7,짜장면,짜장면,간짜장
5,6912,60대 이상,7,비,22.4,28.8,짜장면,짬뽕,짜장면
6,6558,60대 이상,9,맑음,22.5,29.2,후라이드치킨,후라이드치킨,간짜장
7,11622,30대,9,맑음,18.9,26.8,반반치킨,양념치킨,반반치킨
8,9467,60대 이상,9,비,20.8,25.1,짜장면,짜장면,짬뽕
9,10981,20대,1,맑음,-3.9,7.2,후라이드치킨,양념치킨,후라이드치킨


In [275]:
# True Positive, False Negative, False Positive 초기화
TP = 0  # 실제 정답이 추천된 경우
FN = 0  # 실제 정답이 추천되지 않은 경우
FP = 0  # 실제 정답이 아닌 것만 추천된 경우

# 각 사용자에 대한 추천 결과 검사
for _, row in result_df.iterrows():
    actual = row['실제 정답']
    recommended = [row['추천메뉴 1'], row['추천메뉴 2']]
    #recommended = [row['추천메뉴 1'], row['추천메뉴 2'], row['추천메뉴 3']]

    if actual in recommended:
        TP += 1
    else:
        FN += 1
        FP += 1  # 실제 정답이 추천 메뉴 중 어느 것에도 포함되지 않은 경우

# 재현율 및 정밀도 계산
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
precision = TP / (TP + FP) if (TP + FP) > 0 else 0

print("TP:", TP)
print("FN:", FN)
print("FP:", FP)
print("------------------")
print(f"재현율: {recall:.2f}")
print(f"정밀도: {precision:.2f}")

TP: 10
FN: 0
FP: 0
------------------
재현율: 1.00
정밀도: 1.00
